In [1]:
import torch

In [2]:
maping = lambda x: torch.exp(-x)

In [3]:
X = torch.rand((100, 2))
Y = torch.sum(maping(X), dim=1)

In [4]:
X.shape, Y.shape

(torch.Size([100, 2]), torch.Size([100]))

In [ ]:
class Dense_layer:
    def __init__(self, n_neurons, lr=1):
        self.n_neurons = n_neurons
        self.has_initialized = False
        self.lr = lr
        
    def forwardPass(self, input_):
        if self.has_initialized is False:
            self.weights = torch.rand(n_neurons, input_.shape[1])
            self.bias = torch.rand(torch.Size([]))
            self.has_initialized = True
        self.input = input_.requires_grad_()
        self.output = (self.weights @ self.input.T + self.bias).T
        
    def backwardPropogation(self):
        self.output.backward()
        
    def update(self):
        self.input.data -= self.input.grad.data * lr
        self.input.grad = None